In [2]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import MultiDiscrete, Discrete

In [3]:
print(gym.__version__)

0.28.1


# Creating a Custom Environment



In [5]:
# make an environment
import random
import numpy as np
class SchedulerEnvironment(gym.Env):
    def __init__(self, scenario_one = True, terminate_num = 200, mean_delay_one = 6, mean_delay_two = 4, margin_of_delay = 1):
        self.scenario_one = scenario_one
        self.mean_delay_one = mean_delay_one
        self.mean_delay_two = mean_delay_two
        self.margin_of_delay = margin_of_delay


        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 

        self.incoming_packets = dict(
            {
                0: self.queue_one_incoming_packet,
                1: self.queue_two_incoming_packet,
                2: self.queue_best_effort_incoming_packet

            }
            )


        self.step_counter = 0 # i.e. timeslots
        self.terminate_num = terminate_num

        # maximum size of the queue
        self.max_queue_size = 10
        
        # can adjust size of max delay in timeslots after initial observations
        MAX_DELAY = 100

        queue_sample = np.arange(self.max_queue_size)
        for i in range(self.max_queue_size):
            queue_sample[i] = MAX_DELAY

        self.observation_space = spaces.Dict( 
            {
                
                "queues": MultiDiscrete(queue_sample), 
                                
#                 boolean for timeslot delay during switch 
#                 zero for no delay, one for delay. 
                "switchCounter": Discrete(2)
                
            }
        )

        self.queue_one_delay = [] 

        self.queue_two_delay = []

        self.queue_best_effort_delay = [] 

        self.queue_delays = dict(
            {
                0: self.queue_one_delay,
                1: self.queue_two_delay,
                2: self.queue_best_effort_delay
            }
        )

        self.queue_one_removed = 0
        self.queue_two_removed = 0
        self.queue_best_effort_removed = 0

        self.removed_packets = dict(
            {
                0: self.queue_one_removed,
                1: self.queue_two_removed,
                2: self.queue_best_effort_removed
            }
        )

        self.current_state = None
        
        self.action_space = spaces.Discrete(3)
        
        self._action_to_queue = {
                0: self.queue_delays[0],
                1: self.queue_delays[1],
                2: self.queue_delays[2]
            }

    def _increment_delay(self):
        for i in range(len(self.queue_delays)):    
            for j in range(len(self.queue_delays[i])):
                self.queue_delays[i][j] += 1

    def _can_add_packets(self, packet_arrival, action ):
        # increment before evaluating arrival. 
        self.incoming_packets[action] += packet_arrival
        packet_arrived = self.incoming_packets[action] >= 1.0
        not_max_size = len(self.queue_delays[action]) < self.max_queue_size

        if packet_arrived and not_max_size:
            self.incoming_packets[action] -= 1.0
            self.queue_delays[action].append(0)                     

    def _add_packets(self):   
        PACKET_ARRIVAL_ONE = 0.3
        PACKET_ARRIVAL_TWO = 0.25
        PACKET_ARRIVAL_BEST_EFFORT = 0.4
        
        self._can_add_packets(PACKET_ARRIVAL_ONE, 0)
        self._can_add_packets(PACKET_ARRIVAL_TWO, 1)
        self._can_add_packets(PACKET_ARRIVAL_BEST_EFFORT, 2)

        return
     
    def _modify_states(self):
        # will not calculate delay for freshly added packets. 
        self._increment_delay()
        self._add_packets()
        return 
    
    def _get_obs(self):
        return { 
            "queues": self.queue_delays
            }
    
    def _calculate_avg_delay(self, queue):
        queue_size = len(queue)
        if (queue_size == 0):
            return 0
        
        # sum
        sum = 0
        for i in range(queue_size):
            sum += queue[i]
        
        # avg
        avg = sum / queue_size
        return avg

    def _get_info(self):

        avg_one = self._calculate_avg_delay(self.queue_delays[0])
        avg_two = self._calculate_avg_delay(self.queue_delays[1])
        avg_best = self._calculate_avg_delay(self.queue_delays[2])

        # return average delays for each queue. 
        avg_mean_all_queues = [avg_one, avg_two, avg_best]
        
        # return the number of packets removed in each queue. 
        
        packets_removed = [self.removed_packets[0], self.removed_packets[1], self.removed_packets[2]]

        # return the queue num.
        return [avg_mean_all_queues, packets_removed, self.step_counter + 1]
 
    def _initialise_delays(self, queue):
        # give a random delay between 0 and 5
        size_of_queue = len(queue)
        for i in range(size_of_queue):
            queue[i] = random.randint(0, 5)

    def _set_all_zero(self):
        for i in range((3)):
            self.queue_delays[i].clear()
            self.removed_packets[i] = 0

        self.step_counter = 0
        self.current_state = None
        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 
    
    def reset(self, seed=None, options=None):
        # reset everything 
        self._set_all_zero()

        # set the size of the queue. 
        self.queue_delays[0] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[1] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[2] = self.observation_space["queues"].sample().tolist()

        # set seed before randomising 
        random.seed(seed)

        self._initialise_delays(self.queue_delays[0])
        self._initialise_delays(self.queue_delays[1])
        self._initialise_delays(self.queue_delays[2])

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def _reward_function(self, action, empty_queue):
        if empty_queue:
            return -1
        
        reward = 0

        mean_delays = dict(
            {
                0: self.mean_delay_one,
                1: self.mean_delay_two
            }
        )

        if (action == 2):
        
            # reward taking the best effort queue when mean delays are adjusted
            # for priority queues.
            avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
            within_margin_first_queue = ( avg_first_queue <= mean_delays[0] + self.margin_of_delay)
            
            avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
            within_margin_second_queue = ( avg_second_queue <= mean_delays[1] + self.margin_of_delay)
            
            if (within_margin_first_queue and within_margin_second_queue):
                reward = 1

            # Punish for not prioritising. 
            else:
                reward = -1

        else:
            avg_delay = self._calculate_avg_delay(self.queue_delays[0])
            # packet delay > mean delay by the margin
            if (mean_delays[action] + self.margin_of_delay < avg_delay):
                # encourage minimising the delay for this queue.
                reward = 0

            # packet delay < mean delay by the margin
            elif (mean_delays[action] - self.margin_of_delay > avg_delay):
                # discourage minimising the delay too much.
                reward = -1

            # packet delay within margin, don't perform again. 
            else:
                reward = -1

        return reward 

    
    def _remove_packet(self, action):
        # retrieve the packet. 
        self.queue_delays[action].pop(0)
        self.removed_packets[action] += 1

    def step(self, action):
        empty_queue = False
        if len(self.queue_delays[action]) == 0:
            empty_queue = True

        else:
            if self.scenario_one:
                self._remove_packet(action)
    
            else: 
                if (action == self.current_state):
                    self._remove_packet(action)


        # perform queue switch given the conditions.
        if not self.scenario_one:
            if (action != self.current_state):
                self.current_state = action
        
        reward = self._reward_function(action, empty_queue)
                
        self._modify_states()

        info = self._get_info()

        terminated = False
        if self.step_counter + 1 == self.terminate_num:
            terminated = True
        self.step_counter += 1

        # observation made after modifying states. 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


In [6]:
env = SchedulerEnvironment(terminate_num = 50) 

In [12]:
def print_info(info):
    print("----------")
    print("current step =", info[2])
    print("-----")
    print("Average delay of first queue: {:.2f}".format(info[0][0]))
    print("Average delay of second queue: {:.2f}".format(info[0][1]))
    print("Average delay of best-effort queue: {:.2f}".format(info[0][2]))
    print("-----")
    print("Packets removed from first queue:", info[1][0])
    print("Packets removed from second queue:", info[1][1])
    print("Packets removed from best-effort queue:", info[1][2])
    

In [13]:
def print_observation(observation):
    print("----------")
    print("First queue:", observation["queues"][0])
    print("Second queue:", observation["queues"][1])
    print("Best effort queue:", observation["queues"][2])
    

In [14]:
env.action_space.sample()

2

In [15]:
obs, info = env.reset()
print_observation(obs)
print_info(info)

----------
First queue: [3, 0, 0, 3, 1, 3, 5, 4, 0, 4]
Second queue: [2, 5, 2, 5, 4, 4, 1, 3, 1, 1]
Best effort queue: [5, 4, 3, 0, 4, 4, 5, 2, 5, 3]
----------
current step = 1
-----
Average delay of first queue: 2.30
Average delay of second queue: 2.80
Average delay of best-effort queue: 3.50
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 0


In [16]:
def random_scheduler(seed = None):
    terminated = False 
    truncated = False

    # reset seeds the randomiser
    env.reset(seed)
    score = 0

    while not terminated:  
    #     random 
        action = random.randint(0, 2)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [18]:
random_scheduler(0)

----------
First queue: [4, 4, 1, 3, 5, 4, 4, 3, 4, 3]
Second queue: [5, 2, 5, 2, 3, 2, 1, 5, 3, 5]
Best effort queue: [5, 2, 3, 1, 6, 1, 6, 3, 4, 0]
----------
current step = 1
-----
Average delay of first queue: 3.50
Average delay of second queue: 3.30
Average delay of best-effort queue: 3.10
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [5, 2, 4, 6, 5, 5, 4, 5, 4]
Second queue: [6, 3, 6, 3, 4, 3, 2, 6, 4, 6]
Best effort queue: [6, 3, 4, 2, 7, 2, 7, 4, 5, 1]
----------
current step = 2
-----
Average delay of first queue: 4.44
Average delay of second queue: 4.30
Average delay of best-effort queue: 4.10
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [6, 3, 5, 7, 6, 6, 5, 6, 5]
Second queue: [4, 7, 4, 5, 4, 3, 7, 5, 7, 0]
Best effort queue: [7, 4, 5, 3, 8, 3, 8, 5, 6, 2]
----------
current ste

In [19]:
random_scheduler(1)

----------
First queue: [2, 5, 1, 3, 1, 4, 4, 4, 6, 4]
Second queue: [2, 1, 4, 1, 4, 4, 5, 1, 6, 4]
Best effort queue: [6, 2, 5, 1, 3, 1, 1, 1, 6, 0]
----------
current step = 1
-----
Average delay of first queue: 3.40
Average delay of second queue: 3.20
Average delay of best-effort queue: 2.60
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [6, 2, 4, 2, 5, 5, 5, 7, 5]
Second queue: [3, 2, 5, 2, 5, 5, 6, 2, 7, 5]
Best effort queue: [7, 3, 6, 2, 4, 2, 2, 2, 7, 1]
----------
current step = 2
-----
Average delay of first queue: 4.56
Average delay of second queue: 4.20
Average delay of best-effort queue: 3.60
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [7, 3, 5, 3, 6, 6, 6, 8, 6]
Second queue: [3, 6, 3, 6, 6, 7, 3, 8, 6]
Best effort queue: [8, 4, 7, 3, 5, 3, 3, 3, 8, 2]
----------
current step =

In [20]:
random_scheduler(2)

----------
First queue: [1, 1, 1, 3, 2, 6, 6, 3, 3, 5]
Second queue: [5, 1, 5, 6, 2, 4, 6, 4, 6]
Best effort queue: [5, 3, 5, 4, 5, 3, 1, 1, 3, 4]
----------
current step = 1
-----
Average delay of first queue: 3.10
Average delay of second queue: 4.33
Average delay of best-effort queue: 3.40
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [2, 2, 2, 4, 3, 7, 7, 4, 4, 6]
Second queue: [2, 6, 7, 3, 5, 7, 5, 7, 0]
Best effort queue: [6, 4, 6, 5, 6, 4, 2, 2, 4, 5]
----------
current step = 2
-----
Average delay of first queue: 4.10
Average delay of second queue: 4.67
Average delay of best-effort queue: 4.40
-----
Packets removed from first queue: 0
Packets removed from second queue: 2
Packets removed from best-effort queue: 0
----------
First queue: [3, 3, 3, 5, 4, 8, 8, 5, 5, 7]
Second queue: [7, 8, 4, 6, 8, 6, 8, 1]
Best effort queue: [7, 5, 7, 6, 7, 5, 3, 3, 5, 6]
----------
current step = 3


In [21]:
def is_empty(queue):
    if (queue == []):
        return True
    else:
        return False

In [22]:
# FIFO
def fifo_action(obs):
    first_queue = obs["queues"][0]
    second_queue = obs["queues"][1]
    best_effort_queue = obs["queues"][2]
    
    first_queue_element = None
    second_queue_element = None
    best_effort_queue_element = None
    
    if (not is_empty(first_queue)):
        first_queue_element = first_queue[0]
    
    if (not is_empty(second_queue_element)):
        second_queue_element = second_queue[0]
    
    if (not is_empty(best_effort_queue)):
        best_effort_queue_element = best_effort_queue[0]
    
    largest_delay = max(first_queue_element, second_queue_element, best_effort_queue_element)
    
    action = 0
    if (largest_delay == best_effort_queue_element):
        action = 2
    elif (largest_delay == second_queue_element):
        action = 1
    
    return action

In [23]:
def fifo_scheduler(seed = None):
    terminated = False 
    truncated = False
    obs, info = env.reset(seed)
    score = 0
    while not terminated: 

    #     fifo
        action = fifo_action(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [24]:
fifo_scheduler(0)

----------
First queue: [4, 4, 1, 3, 5, 4, 4, 3, 4, 3]
Second queue: [5, 2, 5, 2, 3, 2, 1, 5, 3, 5]
Best effort queue: [5, 2, 3, 1, 6, 1, 6, 3, 4, 0]
----------
current step = 1
-----
Average delay of first queue: 3.50
Average delay of second queue: 3.30
Average delay of best-effort queue: 3.10
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [5, 5, 2, 4, 6, 5, 5, 4, 5, 4]
Second queue: [6, 3, 6, 3, 4, 3, 2, 6, 4, 6]
Best effort queue: [3, 4, 2, 7, 2, 7, 4, 5, 1, 0]
----------
current step = 2
-----
Average delay of first queue: 4.50
Average delay of second queue: 4.30
Average delay of best-effort queue: 3.50
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [6, 6, 3, 5, 7, 6, 6, 5, 6, 5]
Second queue: [4, 7, 4, 5, 4, 3, 7, 5, 7]
Best effort queue: [4, 5, 3, 8, 3, 8, 5, 6, 2, 1]
----------
current 

In [125]:
fifo_scheduler(1)

----------
First queue: [2, 5, 1, 3, 1, 4, 4, 4, 6, 4]
Second queue: [2, 1, 4, 1, 4, 4, 5, 1, 6, 4]
Best effort queue: [6, 2, 5, 1, 3, 1, 1, 1, 6, 0]
----------
current step = 1
-----
Average delay of first queue: 3.40
Average delay of second queue: 3.20
Average delay of best-effort queue: 2.60
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [3, 6, 2, 4, 2, 5, 5, 5, 7, 5]
Second queue: [3, 2, 5, 2, 5, 5, 6, 2, 7, 5]
Best effort queue: [3, 6, 2, 4, 2, 2, 2, 7, 1, 0]
----------
current step = 2
-----
Average delay of first queue: 4.40
Average delay of second queue: 4.20
Average delay of best-effort queue: 2.90
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [4, 7, 3, 5, 3, 6, 6, 6, 8, 6]
Second queue: [4, 3, 6, 3, 6, 6, 7, 3, 8, 6]
Best effort queue: [7, 3, 5, 3, 3, 3, 8, 2, 1, 0]
----------
curre

In [126]:
fifo_scheduler(2)

----------
First queue: [1, 1, 1, 3, 2, 6, 6, 3, 3, 5]
Second queue: [2, 5, 1, 5, 6, 2, 4, 6, 4, 6]
Best effort queue: [3, 5, 4, 5, 3, 1, 1, 3, 4, 0]
----------
current step = 1
-----
Average delay of first queue: 3.10
Average delay of second queue: 4.10
Average delay of best-effort queue: 2.90
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [2, 2, 2, 4, 3, 7, 7, 4, 4, 6]
Second queue: [3, 6, 2, 6, 7, 3, 5, 7, 5, 7]
Best effort queue: [6, 5, 6, 4, 2, 2, 4, 5, 1, 0]
----------
current step = 2
-----
Average delay of first queue: 4.10
Average delay of second queue: 5.10
Average delay of best-effort queue: 3.50
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [3, 3, 3, 5, 4, 8, 8, 5, 5, 7]
Second queue: [4, 7, 3, 7, 8, 4, 6, 8, 6, 8]
Best effort queue: [6, 7, 5, 3, 3, 5, 6, 2, 1, 0]
----------
curre

FIFO tries to minimise all queues, which means the average delays for each queue converges. 

In [119]:
class EDFScheduler(): 
    
    def __init__(self, scheduler_environment):
        self.edf_env = scheduler_environment
        self.current_packet_removed_one = 0
        self.current_packet_removed_two = 0
        self.current_packet_removed_best = 0
        
#         random intialisation values picked with the same mean delays. 
        self.deadline_one = 6
        self.deadline_two = 4
        self.deadline_best_effort = 10
        
        self.deadlines = np.array([self.deadline_one, self.deadline_two, self.deadline_best_effort])
    
#     obs must be the observation returned by schedulerEnvironment. 
    def _edf_action(self, mean_delay_one, mean_delay_two, mean_delay_best_effort, info):
        
        expected_mean_delays = [mean_delay_one, mean_delay_two, mean_delay_best_effort]
        
#         random initial values. 
        mean_delay_one_increment = 6
        mean_delay_two_increment = 4
        mean_delay_best_effort_increment = 10
        
        increments = [mean_delay_one_increment, mean_delay_two_increment, mean_delay_best_effort_increment]

#         ideally we only want to change the queue that was changed.
#         how to obtain num? using get info we retrieve the previous packets + packets removed.
        old_packet_removed_one = self.current_packet_removed_one
        old_packet_removed_two = self.current_packet_removed_two
        old_packet_removed_best = self.current_packet_removed_best
        
#         info[1][num] = number of removed packets for queue num. 
        self.current_packet_removed_one = info[1][0]
        self.current_packet_removed_two = info[1][1]
        self.current_packet_removed_best = info[1][2]
        
        difference = np.array([self.current_packet_removed_one - old_packet_removed_one])
        difference = np.append(difference, self.current_packet_removed_two - old_packet_removed_two)
        difference = np.append(difference, self.current_packet_removed_best - old_packet_removed_best)
        
#         the queue which the next deadline is to be decided
        queue_deadline = np.argmax(difference, axis = 0)
    
#         increment deadlines given the conditions: mean delay < expected delay.  
        if (info[0][queue_deadline] < expected_mean_delays[queue_deadline]):
#         increment by the amount which the mean delay was off from the expected delay.
            self.deadlines[queue_deadline] += increments[queue_deadline] 
            
#         constant that modifies the level which the difference between mean delays is increased.
#         this factor is multiplication as the values mean values are small for queue 1 and 2. 
            FACTOR_DEADLINE = 3
            increment_deadline = (expected_mean_delays[queue_deadline] - info[0][queue_deadline]) * FACTOR_DEADLINE
            self.deadlines[queue_deadline] += increment_deadline
            
        else:
#             increment normally.
            self.deadlines[queue_deadline] += increments[queue_deadline]
            
#     choose the smallest deadline to return
        return np.argmin(self.deadlines, axis = 0)
    
#     mean_delay_one += mean_delay_one_increment
    def perform_scheduling(self, mean_delay_one, mean_delay_two, mean_delay_best_effort, seed = None):
        terminated = False 
        truncated = False
        obs, info = self.edf_env.reset(seed)
        score = 0
        while not terminated: 

        #     edf
            action = self._edf_action(mean_delay_one, mean_delay_two, mean_delay_best_effort, info)
            obs, reward, terminated, truncated, info = self.edf_env.step(action)
            score += reward
            print_observation(obs)
            print_info(info)
        print(score)
    

In [120]:
# predefine the expected mean delay for first queue, second queue and best effort queue. 
mean_delay_one = 6
mean_delay_two = 4
mean_delay_best_effort = 10

edf_env = SchedulerEnvironment(terminate_num = 50, mean_delay_one = mean_delay_one, mean_delay_two = mean_delay_two)


In [121]:
edf_scheduler = EDFScheduler(edf_env)

In [122]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 0)

----------
First queue: [4, 4, 1, 3, 5, 4, 4, 3, 4, 3]
Second queue: [2, 5, 2, 3, 2, 1, 5, 3, 5]
Best effort queue: [6, 5, 2, 3, 1, 6, 1, 6, 3, 4]
----------
current step = 1
-----
Average delay of first queue: 3.50
Average delay of second queue: 3.11
Average delay of best-effort queue: 3.70
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [5, 5, 2, 4, 6, 5, 5, 4, 5, 4]
Second queue: [6, 3, 4, 3, 2, 6, 4, 6]
Best effort queue: [7, 6, 3, 4, 2, 7, 2, 7, 4, 5]
----------
current step = 2
-----
Average delay of first queue: 4.50
Average delay of second queue: 4.25
Average delay of best-effort queue: 4.70
-----
Packets removed from first queue: 0
Packets removed from second queue: 2
Packets removed from best-effort queue: 0
----------
First queue: [6, 6, 3, 5, 7, 6, 6, 5, 6, 5]
Second queue: [7, 4, 5, 4, 3, 7, 5, 7]
Best effort queue: [7, 4, 5, 3, 8, 3, 8, 5, 6, 0]
----------
current step = 3
---

In [123]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 1)

----------
First queue: [5, 1, 3, 1, 4, 4, 4, 6, 4]
Second queue: [2, 1, 4, 1, 4, 4, 5, 1, 6, 4]
Best effort queue: [3, 6, 2, 5, 1, 3, 1, 1, 1, 6]
----------
current step = 1
-----
Average delay of first queue: 3.56
Average delay of second queue: 3.20
Average delay of best-effort queue: 2.90
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 0
----------
First queue: [6, 2, 4, 2, 5, 5, 5, 7, 5]
Second queue: [2, 5, 2, 5, 5, 6, 2, 7, 5, 0]
Best effort queue: [4, 7, 3, 6, 2, 4, 2, 2, 2, 7]
----------
current step = 2
-----
Average delay of first queue: 4.56
Average delay of second queue: 3.90
Average delay of best-effort queue: 3.90
-----
Packets removed from first queue: 1
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [7, 3, 5, 3, 6, 6, 6, 8, 6]
Second queue: [6, 3, 6, 6, 7, 3, 8, 6, 1]
Best effort queue: [5, 8, 4, 7, 3, 5, 3, 3, 3, 8]
----------
current step = 3


In [124]:
edf_scheduler.perform_scheduling(mean_delay_one, mean_delay_two, mean_delay_best_effort, 2)

----------
First queue: [1, 1, 3, 2, 6, 6, 3, 3, 5]
Second queue: [2, 5, 1, 5, 6, 2, 4, 6, 4, 6]
Best effort queue: [5, 3, 5, 4, 5, 3, 1, 1, 3, 4]
----------
current step = 1
-----
Average delay of first queue: 3.33
Average delay of second queue: 4.10
Average delay of best-effort queue: 3.40
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 0
----------
First queue: [2, 2, 4, 3, 7, 7, 4, 4, 6]
Second queue: [6, 2, 6, 7, 3, 5, 7, 5, 7]
Best effort queue: [6, 4, 6, 5, 6, 4, 2, 2, 4, 5]
----------
current step = 2
-----
Average delay of first queue: 4.33
Average delay of second queue: 5.33
Average delay of best-effort queue: 4.40
-----
Packets removed from first queue: 1
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [3, 3, 5, 4, 8, 8, 5, 5, 7]
Second queue: [3, 7, 8, 4, 6, 8, 6, 8]
Best effort queue: [7, 5, 7, 6, 7, 5, 3, 3, 5, 6]
----------
current step = 3
-----


We can see that the EDF is already performing much better than the random queue. 
We infer that the EDF scheduler attempts to keep the second queue very small. 